In [ ]:
import torch
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
from pylab import *
import torchvision

### Function to make a dataloader

In [ ]:
def get_data_loader(img_dir, image_size=128, batch_size=16, num_workers=0):
    
    transform = transforms.Compose([transforms.Resize(image_size),
                                    transforms.ToTensor()])
    
    dataset = datasets.ImageFolder(img_dir, transform)
    
    data_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size,
                                              shuffle=True, num_workers=num_workers)
    return data_loader


In [ ]:
data_path = r"C:\Users\manou\Downloads\processed-celeba-small\processed_celeba_small"

data_loader = get_data_loader(data_path)

### Displays images
To make sure the data was batched correctly

In [ ]:
def imshow(img):
    plt.imshow(np.transpose(img.numpy(), (1,2,0)))

dataiter = iter(data_loader)
images, _ = dataiter.next()

fig = plt.figure(figsize=(12,8))
imshow(torchvision.utils.make_grid(images))

### Scales images
In many GAN papers, they use a tanh activation function because it tends to perform the best. This means the data must be scaled  [-1, 1]  instead of  [0, 1]

In [ ]:
def scale(x, feature_range=(-1,1)):
    min, max = feature_range
    x = x*(max-min)+min
    return x

In [ ]:
print('Max :', images[0].max())
print('Min :', images[0].min())

print('-'*24)
im_scaled = scale(images[0])

print('Max :', im_scaled.max())
print('Min :', im_scaled.min())

### Creates the discriminator and generator

In [ ]:
from torch import nn

class Discriminator(nn.Module):
    def __init__(self, conv_dim=64):
        super(Discriminator, self).__init__()
        
        # Defines the convolutional and normalization layers
        self.conv1 = nn.Conv2d(3, conv_dim, 4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(conv_dim, conv_dim*2, 4, stride=2, padding=1)
        self.norm1 = nn.BatchNorm2d(conv_dim*2)
        self.conv3 = nn.Conv2d(conv_dim*2, conv_dim*4, 4, stride=2, padding=1)
        self.norm2 = nn.BatchNorm2d(conv_dim*4)
        self.fc = nn.Linear(256*16*16, 1)
        
    def forward(self, x):
        x0 = x.size(0)
        #print(x.shape)
        x = F.leaky_relu(self.conv1(x))
        #print(x.shape)
        x = F.leaky_relu(self.norm1(self.conv2(x)))
        #print(x.shape)
        x = F.leaky_relu(self.norm2(self.conv3(x)))
        #print(x.shape)
        x = x.view(x0, -1)
        #print(x.shape)
        x = self.fc(x)
        #print(x.shape)
        #print()
        return x

In [ ]:
class Generator(nn.Module):
    def __init__(self, z_size=100, conv_dim=64):
        super(Generator, self).__init__()
        # sets variable
        self.conv_dim = conv_dim
        
        ## generator layers
        # fc layer
        self.linear = nn.Linear(z_size, 256*16*16)
        
        # every other layer
        self.trans1 = nn.ConvTranspose2d(conv_dim*4, conv_dim*2, 4,
                                         stride=2, padding=1)
        self.norm1 = nn.BatchNorm2d(conv_dim*2)
        self.trans2 = nn.ConvTranspose2d(conv_dim*2, conv_dim, 4,
                                         stride=2, padding=1)
        self.norm2 = nn.BatchNorm2d(conv_dim)
        self.trans3 = nn.ConvTranspose2d(conv_dim, 3, 4,
                                       stride=2, padding=1)
        
    def forward(self, x):
        #print(x.shape)
        x = self.linear(x)
        #print(x.shape)
        x = x.view(-1, self.conv_dim*4, 16, 16)
        #print(x.shape)
        x = F.leaky_relu(self.norm1(self.trans1(x)))
        #print(x.shape)
        x = F.leaky_relu(self.norm2(self.trans2(x)))
        #print(x.shape)
        x = F.tanh(self.trans3(x))
        #print(x.shape)
        #print()
        
        return x

In [ ]:
D = Discriminator()
G = Generator()

train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    G.cuda()
    D.cuda()
    print('gpu babyyyyy!')
else:
    G.cpu()
    D.cpu()
    print('training on cpu')
    
print(D)
print()
print(G)

### Defines loss functions

In [ ]:
def real_loss(d_out, smooth=False):
    batch_size = d_out.size(0)
    if smooth:
        labels = torch.ones(batch_size)*0.9
    else:
        labels = torch.ones(batch_size)
    if train_on_gpu:
        labels = labels.gpu()
    
    criterion = nn.BCEWithLogitsLoss()
    loss = criterion(d_out.squeeze(), labels)
    return loss

def fake_loss(d_out):
    batch_size = d_out.size(0)
    labels = torch.zeros(batch_size)
    if train_on_gpu:
        labels = labels.gpu()
        
    criterion = nn.BCEWithLogitsLoss()
    loss = criterion(d_out.squeeze(), labels)
    return loss

### Defines optimizers

In [ ]:
from torch import optim

lr = 0.0002
beta1 = 0.5
beta2 = 0.99

d_optimizer = optim.Adam(D.parameters(), lr, [beta1, beta2])
g_optimizer = optim.Adam(G.parameters(), lr, [beta1, beta2])

### Trains the GAN

In [ ]:
def Train(dataloader, D, G, num_epochs, z_size=100):
    samples = []
    losses = []
    
    print_every = 300
    
    sample_size = 16
    fixed_z = np.random.uniform(-1, 1, size=(sample_size, z_size))
    fixed_z = torch.from_numpy(fixed_z).float()
    
    for e in range(num_epochs):
        for batch_i, (real_images, _) in enumerate(dataloader):
            batch_size = real_images[0].size(0)
            
            real_images = scale(real_images)
            
            if train_on_gpu:
                real_images = real_images.cuda()
                
            # =========================== #
            #        Discriminator        #
            # =========================== #
            
            d_optimizer.zero_grad()
            
            # real image loss
            d_out_real = D(real_images)
            loss_real = real_loss(d_out_real)
            
            # generate image
            z = np.random.uniform(-1,1, size=(batch_size, z_size))
            z = torch.from_numpy(z).float()
            if train_on_gpu:
                z = z.cuda()
            
            # fake image loss
            gen_im = G(z)
            d_out_fake = D(gen_im)
            loss_fake = fake_loss(d_out_fake)
            
            # backprop and step
            d_loss = loss_fake + loss_real
            d_loss.backward()
            d_optimizer.step()
            
            # ========================== #
            #         Generator          #
            # ========================== #
            
            g_optimizer.zero_grad()
            
            # generate image
            z = np.random.uniform(-1,1, size=(batch_size, z_size))
            z = torch.from_numpy(z).float()
            if train_on_gpu:
                z = z.cuda()
                
            # generate image and run thru discriminator
            gen_im = G(z)
            d_out_gen = D(gen_im)
            
            # loss, backprop, step
            g_loss = real_loss(d_out_gen)
            g_loss.backward()
            g_optimizer.step()
            
            if batch_i % print_every == 0:
                losses.append((d_loss.item(), g_loss.item()))
                print('Epoch [{:4d}/{:4d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format(
                    e+1, num_epochs, d_loss.item(), g_loss.item()))
                
            G.eval()
            if train_on_gpu:
                fixed_z = fixed_z.cuda()
            samples_z = G(fixed_z)
            samples.append(samples_z)
            G.train()
    with open('samples.pkl', 'wb') as f:
        pkl.dump(samples, f)

In [ ]:
Train(data_loader, D=D, G=G, num_epochs=5)